# 차수 효율화 프로젝트

### 목표 

차수 배분에 대한 최적 효율화를 통해 더 높은 생산성 유도. 

### 얻는 이점

1. 1차수에 들어갈 최적의 주문건수 의사결정을 내리도록 도움
2. DAS를 이용하지 않아도 되는 주문은 배제시킴


## 문제

차수를 어떻게 효율적으로 나눌것인가가 문제. <br>
90개의 Cells 이 있다고 가정을 했을때, 90개의 주문을 1차수에 넣으면 모든 물품은 문제없이 소화 가능<br>
하지만 1차수에 90개 이상의 주문을 넣었을때 더 효율적으로 상품을 가져올수 있는 기회를 잃어버림. 

1차수에 너무 많은 주문을 넣었다면 (예를 들어 200개의 주문을 1차수에 넣음) DAS 분배과정에서 <br>
Cells있지 않은 상품을 선택하여 생기는 비효율성이 생길수 있음. 

결국 그 둘 사이의 최적점을 찾아서 1차수에 들어갈수 있는 최적의 "주문" 갯수를 알아내는 것이 중요. 

## 알고리즘 


### 배재

1. DAS를 이용할 필요가 없는 주문건들에 관해서 배재를 시킴
2. 주로 한번만 물품을 찾아오면 바로 배송처리가능한 물품들을 배재시킴

### 주문간의 유사도 판별

문제의 유사도 판별은 probability model 그리고 sigmoid activation을 사용함. <br>

$$ H(A, B) = \frac{1}{1+e^{-\sum A주문 * B주문}} $$ 

예를 들어 주문 A에는 a상품 3개, b2개 라는 상품을 갖고 있고, <br>
B라는 주문애는 a상품 3개, b상품 2개, c상품 1개 라는 상품을 갖고 있으며, <br>
C라는 주문에는 c상품 1개, d상품 2개를 갖고 있음 

$$ \begin{align}
A &= \{ 3a, 2b \} \\ 
B &= \{ 2a, 2b, 1c \} \\
C &= \{ 1c, 2d \}
\end{align} $$ 

위의 주문은 선형대수학에서 말하는 vector의 형태로 전환이 가능. 

$$ \begin{align}
A &= [ 3, 2, 0, 0] \\ 
B &= [ 2, 2, 1, 0] \\
C &= [ 0, 0, 1, 2]
\end{align} $$ 

vectors로 표현된 주문들은 서로간의 유사도 판별이 가능해지며 유사도 수치에 따라서 어떤 상품을 같은 차수에 넣을지 결정을 할 수 있음. <br>
아래는 서로 유사도를 비교하는 예제 코드. 

즉 결론적으로 서로 주문들간의 유사도가 높으면 높을수록 1에 가까워지는 확률을 보이며, <br>
서로 상관관계가 전혀 없을시에는 0에 가까운 값을 내놓음

아래의 예제에서는 A와 B의 유사도 0.809는 A 와 C의 유사도 0.5값보다 높다. 

In [3]:
import numpy as np

MAXIMUM = 3

def sigmoid(x):
    return 1/(1 + np.nan_to_num(np.e**(-x)))


def forward(a, b):
    a = a/MAXIMUM
    b = b/MAXIMUM
    return np.sum(a * b)

def process(a, b):
    return sigmoid(forward(a, b))

A = np.array([3, 2, 0, 0])
B = np.array([3, 2, 1, 0])
C = np.array([0, 0, 1, 2])

r1 = process(A, B) # 0.999997739676
r2 = process(A, C) # 0.5
r3 = process(B, C) # 0.527749235055


print('H(A, B) =', r1)
print('H(A, C) =', r2)
print('H(B, C) =', r3)

H(A, B) = 0.809141955755
H(A, C) = 0.5
H(B, C) = 0.527749235055


### 우선순위 문제 - 주문 유사도 분석에 의한 추가 (1차에서 수정됨)

셀의 전체 갯수가 4개라고 가정을 하고, 현재 모두 다 가득차 있는 상태라면 다음과 같이 그려질수 있음 

$$
\text{1번 셀} = [4, 2, 0, 0, 0, 0, 0, 0] \\
\text{2번 셀} = [3, 2, 0, 0, 0, 0, 0, 0] \\
\text{3번 셀} = [0, 0, 1, 2, 0, 0, 0, 0] \\
\text{4번 셀} = [1, 0, 2, 0, 0, 0, 0, 0]
$$

현재 DAS상태를 대표할수 있는 값은 모든 cell vectors를 합한 값. 즉.. 

$$ \text{1번 셀} + \text{2번 셀} + \text{3번 셀} + \text{4번 셀} = [8, 4, 3, 2, 0, 0, 0, 0] $$



이 때 어떤 주문을 넣는것이 (같은 차수안에..) 가장 효율적인가에 대한 문제가 생김<br>
예를 들어서 D, E, F라는 선택 가능한 주문이 있음
(현재 Cell의 4칸 모두 주문이 차 있는 상태) 

$$ \begin{align}
D &= [0, 0, 0, 1, 0, 0, 0, 0] \\ 
E &= [3, 3, 1, 0, 0, 0, 0, 0] \\
F &= [1, 1, 1, 1, 0, 0, 0, 0] \\
G &= [1, 1, 1, 1, 1, 1, 1, 3]
\end{align} $$ 


D, E, F, G중에서 어떤 것을 우선적으로 넣을 것인가가 문제. <br>
이 경우 다음의 공식을 따름

$$ H(A, B) = \frac{1}{\text{N}} \cdot \frac{1}{1+e^{-\sum 전체주문 * \text{추가할주문}}}  $$ 

* 여기서 N의 값은 추가할 주문의 상품갯수<br>
* 즉 <span style="color:red">현재 DAS에 추가된 주문들과 추가할 주문의 유사도 비교가 아닌 <b>해당 추가할 주문의 개별 물품 하나당 유사도를 찾음</b></span>

아래의 예제를 보면 D는 물품이 한건밖에 안되지만 유사도 수치가 0.55에 달하여 가장 높음. <br>
이유는 주문과 주문의 유사도비교가 아닌 물품당 유사도를 비교하기 때문에 <span style="color:red">알고리즘은 물품은 적으면서도 가장 유사도가 높은 주문을 자동으로 찾게 됨</span><br>
E와 F를 비교시 역시 7건의 물품을 갖고 있는 E보다, 4건의 물품을 갖고 있는 F주문이 더 높은 유사도를 보임

In [39]:
def priority(a, b):
    return process(a, b)#/np.sum(b)

R = np.array([8, 4, 3, 2, 0, 0, 0, 0])
D = np.array([0, 0, 0, 1, 0, 0, 0, 0])
E = np.array([3, 3, 1, 0, 0, 0, 0, 0])
F = np.array([1, 1, 1, 1, 0, 0, 0, 0])
G = np.array([1, 1, 1, 1, 1, 1, 1, 3])


print('H(R, D) =', priority(R, D))
print('H(R, E) =', priority(R, E))
print('H(R, F) =', priority(R, F))
print('H(R, G) =', priority(R, G)/4)

H(R, D) = 0.555328055262
H(R, E) = 0.987046272469
H(R, F) = 0.868628790469
H(R, G) = 0.217157197617


In [53]:
def priority(a, b):
    return process(a, b)#/np.sum(b)

R = np.array([8, 4, 3, 2, 0, 0, 0, 0])
D = np.array([0, 1, 1, 1, 0, 0, 0, 0])
E = np.array([3, 3, 1, 0, 0, 0, 0, 0])
F = np.array([1, 1, 1, 1, 0, 0, 0, 0])
G = np.array([1, 1, 1, 1, 1, 1, 1, 3])
H = np.array([1, 1, 1, 1, 1, 0, 1, 0])
I = np.array([1, 1, 1, 1, 1, 0, 0, 0])


print('H(R, D) =', priority(R, D))
print('H(R, E) =', priority(R, E))
print('H(R, F) =', priority(R, F))
print('H(R, G) =', priority(R, G)/4.2)
print('H(R, G) =', priority(R, H)/2.2)
print('H(R, G) =', priority(R, H)/1.2)

H(R, D) = 0.73105857863
H(R, E) = 0.987046272469
H(R, F) = 0.868628790469
H(R, G) = 0.206816378683
H(R, G) = 0.394831268395
H(R, G) = 0.723857325391


In [31]:
a = np.array([3, 3, 3])
b = np.array([3, 0, 3])
c = np.array([2,0])
d = np.array([1,0])
print(priority(a,a))
print(priority(a,b))
# print(priority(a,c))
# print(priority(a,d))

0.952574126822
0.880797077978


### 위치기반에 의한 유사도 분석

만약 주문 유사도로 잡히지 않는 주문건들에 대해서는 위치기반으로 (예 E1-1)을 통하여 유사도 판단후 DAS에 추가시킬지 결정


### 1차수에 얼마만큼의 주문을 넣을지 문제

사실 정답이 없음. <br>
정답을 가장 잘 아는 사람은 바로 현장에서 일하고 계신 분들이 가장 잘 알고 있음. <br>
즉 현장 사람들이 정할수 있도록 옵션을 주는 것이 가장 효율적이라고 판단. 

**즉 해당 비율을 현장에 계신 분들이 정하면 그에 따라서 자동으로 차수를 나눠줌**

$$ \text{비율} = \frac{\text{못들어가는 양}}{\text{들어갈수 있는 양}} $$

예를 들어서 96개 Cells에 들어갈 주문 96개에 대해서 다음과 같이 vector로 나타내면 다음과 같음 

$$ [100, 50, 30, 10, 0, 0, 0] $$


현재 96개의 주문을 1차수에 넣은 상태이지만, 추가적으로 문제없이 다음의 주문들을 추가시킬수 있음<br>
즉 현재 찾은 96개의 주문들과 동일한 상품을 갖은 주문건에 대해서는 문제없이 추가시킴<br>
아래의 추가시켜도 되는 주문들 (D, E, F, G)는 해당물품을 잡고 바코드를 찍었을때 문제없이 DAS에 들어간다고 판단함


$$ \begin{align}
D &= [1, 1, 1, 1, 0, 0, 0, 0] \\ 
E &= [0, 0, 0, 3, 0, 0, 0, 0] \\
F &= [3, 0, 2, 0, 0, 0, 0, 0] \\
G &= [4, 4, 4, 4, 0, 0, 0, 0]
\end{align} $$ 

하지만 다음의 주문건들은 문제가 생길수 있음

$$ \begin{align}
H &= [0, 0, 0, 0, 1, 0, 0, 0] \\ 
I &= [0, 0, 0, 2, 0, 3, 0, 0] \\
J &= [0, 0, 0, 0, 1, 1, 1, 1] \\
K &= [0, 0, 0, 2, 3, 0, 0, 0]
\end{align} $$ 


예를 들어서 현재까지 1차수에 잡힌 총 물품 갯수가 200개라고 하였을때, K를 추가시키면 다음과 같이 공식을 세울수 있음 

$$ \frac{3}{200 + 2} * 100 = 1.4851\% $$

즉 최악의 경우 현재 DAS가 처리해야할 주문들로 꽉 찬 상태이고 어떠한 Cell도 나가지 못하는 상황일때.. <br>
이때 DAS로 넣을 물품을 들어 바코드를 찍었을때 처리하지 못하고 다시 내려놔야 하는 percentage를 가르킴. 

해당 percentage는 현장에 계신 작업자 분들께서 정해주시며 해당 비율에 따라서 1차수에 얼마만큼의 물품이 들어갈지 결정이 됨


> percentage를 정하는 부분까지 최적화 하기 위해서는 해당 작업에 대한 실질적인 데이터가 필요함<br>
> 예를 들어서 몇퍼센트에서 1차수를 끝내는데 몇시간이 걸렸다라는 방대한 데이터가 필요한데<br>
> 사실 이런 부분은 사람의 머리로도 충분히 해결 가능하다고 보고 있음. <br>
> 문제는 최적의 percentage를 찾기 까지 작업자들의 충분한 시도가 필요함
